In [85]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Datos #

### 2.1 Perceptron multicapa ###

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()  # Convierte la imagen de un formato PIL o numpy.ndarray a un tensor.
])

mnist_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)

# Creamos un DataLoader que nos permite cargar los datos en lotes pequeños.
# `dataset=mnist_dataset` es el dataset que se cargará.
# `batch_size=16` indica que cada lote contendrá 16 imágenes y etiquetas.
# `shuffle=True` mezcla los datos aleatoriamente en cada época, mejorando la generalización del modelo.
data_loader = DataLoader(
    mnist_dataset, batch_size=16, shuffle=True
)

# Obtenemos un único lote de datos del DataLoader.
# `next(iter(data_loader))` convierte el DataLoader en un iterador y toma el primer lote.
# El lote contiene `images` (los tensores de las imágenes) y `labels` (las etiquetas correspondientes).
images, labels = next(iter(data_loader))


# Plot the images in a grid
plt.figure(figsize=(10, 10))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(images[i].squeeze(), cmap='gray')
    plt.title(f'Label: {labels[i].item()}')
    plt.axis('off')
plt.tight_layout()
plt.show()

# Arquitectura # 

In [66]:
# para crear, entrenar y evaluar redes neuronales.
class MLP(nn.Module):
    def __init__(self):
        # Inicializamos la clase base nn.Module
        # Esto habilita funciones esenciales como la gestión de capas y forward pass.
        super(MLP, self).__init__()
        # Capa completamente conectada: de entrada (28x28 píxeles) a 512 neuronas
        self.fc1 = nn.Linear(28 * 28, 512)
        # Capa oculta: de 512 neuronas a 256 neuronas
        self.fc2 = nn.Linear(512, 256)
        # Capa oculta: de 256 neuronas a 128 neuronas
        self.fc3 = nn.Linear(256, 128)
        #Capa de salida: de 128 neuronas a 10 clases (números del 0 al 9)
        self.fc4 = nn.Linear(128, 10)
        # Función de activación Sigmoid
        self.ReLU =  nn.ReLU()
        # Dropout para evitar sobreajuste
        self.dropout = nn.Dropout(0.2)

    # Definimos cómo pasa la información a través de la red
    # Este método es obligatorio en las clases que heredan de nn.Module.
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Aplanamos las imágenes (de 28x28 a 1D)
        x = self.ReLU(self.fc1(x))  # Aplicamos la primera capa y Sigmoid
        x = self.dropout(x)         # Aplicamos Dropout
        x = self.ReLU(self.fc2(x))  # Aplicamos la segunda capa y Tahn
        x = self.dropout(x)         # Aplicamos Dropout
        x = self.ReLU(self.fc3(x))
        x = self.fc4(x)  # Aplicamos la capa de salida
        return x

# Entrenamiento

In [ ]:
# Hiperparámetros
batch_size = 64 # Tamaño de lote
learning_rate = 0.1 # Tasa de aprendizaje
epochs = 10       # Número de épocas de entrenamiento

# Preprocesamiento y carga de datos de MNIST
transform = transforms.Compose([
    transforms.ToTensor(),                 # Convertimos imágenes a tensores
    transforms.Normalize((0.5,), (0.5,))  # Normalizamos a media 0 y varianza 1
])
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)  # Dataset de entrenamiento
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)  # Dataset de prueba
train_loader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)  # Dataloader para entrenamiento
test_loader = DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)  # Dataloader para prueba

# Definimos el modelo, la función de pérdida y el optimizador
model = MLP()                             # Creamos una instancia del modelo MLP
criterion = nn.CrossEntropyLoss()         # Función de pérdida para clasificación
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)  # Optimizador Adam

# Bucle de entrenamiento
for epoch in range(epochs):
    model.train()  # Ponemos el modelo en modo entrenamiento
    for images, labels in train_loader:  # Iteramos sobre lotes de datos
        optimizer.zero_grad()            # Reiniciamos los gradientes
        outputs = model(images)          # Hacemos una predicción con el modelo
        loss = criterion(outputs, labels)  # Calculamos la pérdida
        loss.backward()                  # Propagamos los gradientes
        optimizer.step()                 # Actualizamos los pesos del modelo

    # Mostramos la pérdida al final de cada época
    print(f"Época [{epoch+1}/{epochs}], Pérdida: {loss.item():.4f}")

# Evaluación del modelo

In [ ]:
model.eval()  # Ponemos el modelo en modo evaluación (desactiva Dropout)
correct = 0
total = 0
with torch.no_grad():  # Desactivamos el cálculo de gradientes para evaluación
    for images, labels in test_loader:  # Iteramos sobre los datos de prueba
        outputs = model(images)         # Hacemos predicciones
        _, predicted = torch.max(outputs.data, 1)  # Obtenemos la clase con mayor probabilidad
        total += labels.size(0)         # Total de muestras evaluadas
        correct += (predicted == labels).sum().item()  # Contamos las predicciones correctas

# Calculamos y mostramos la precisión del modelo
accuracy = 100 * correct / total
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}%")

### Datos pruebas ###

1er intento -> sigmoid y adam: valores de pérdida mayores a 1.3 en las 5 épocas. Learning rate es 0.01, batch size 128. -> Accuracy 44.96% 
2do intento -> batch size 64, learning rate 0.001. Pérdida 0.29 es la mayor. -> Accuracy 96.62%
3er intento -> batch size 128, learning rate 0.01, epoch 5. -> Accuracy 90.74% IMPORTANTE VER EL AUMENTO QUE TUVO. 
4to intento -> batch size 64, learning rate 0.01, epoch 5. Mayores valores de pérdida 1.1 el más bajo y alto 1.9. -> Accuracy 63.96%
5to intento -> batch 128, learning rate 0.1, epoch 5. Perdida con valores de 2. -> Accuracy 11.35%

sigmoid y SGD 
1er intento -> batch size 128, learning rate 0.01. Pérdida con valores grandes de 2. -> Accuracy 73.28%
2do intento -> batch 64, learning rate 0.001. Perdida valores de 2. -> Accuracy 11.35%
3r intento -> batch 128, learning 0.2. Perdida menores. -> Accuracy 96.98
4to intento -> batch 128, learning 0.5. Perdida al principio mayores. de 2, pero luego 0.4. -> Accuracy 95.11%

TANH Y SGD
1er intento -> batch 128, learning 0.001. Perdida medias. -> Accuracy 91.90%
2do intento -> 128, learning 0.01. Perdida menos.-> Accuracy 96.68%
3er intento -> 128, learning 0.1, perdida 0.29. -> Accuracy 95%

TANH Y ADAM
1er intento -> batch 128, learning 0.001. Perdida 0.22.-> Accuracy 96.81%
2do intento -> batch 64, learning 0.001. Pérdida 0.28.-> Accuracy 96.50%
3er intento -> batch 64, learning 0.01. Pérdida 2.4.-> Accuracy 10.28%
5to intento -> batch 64, learning 0.1. Pérdida 4.-> Accuracy 10.32%

RELU Y ADAM
1er intento -> batch 64, learning 0.001. Pérdida 0.3.-> Accuracy 97.13%
2do intento -> batch 64, learning 0.01. Pérdida 0.6.-> Accuracy 87.60%
3er intento -> batch 64, learning 0.1. Pérdida 2.3.-> Accuracy 11.35%
4to intento aumenté epoch 10. -> batch 64, learning 0.005. Pérdida 0.42.-> Accuracy 95.50%
5to intento epoch 10 -> batch 64, learning 0.001. Pérdida 0.23. -> Accuracy 97.61%

RELU Y SGD 
1er intento -> batch 64, learning 0.001. Pérdida 0.7. -> Accuracy 96.18% 
2do intento -> batch 64, learning 0.01. Pérdida 0.28. -> Accuracy 97.97% EL MEJOR.
3er intento -> batch 64, learning 0.1. Pérdida 2. -> Accuracy 19.95%

## Informe ##

Luego de los experimentos realizados, el que tuvo mejor accuracy fue la función de activación de Relu con optimizador SGD. EL resultado fue de 97.97%. Cuando se usó un learning rate muy alto el accuracy tendió a bajar mucho y en el caso de los datos que tenemos, de carácter MNIST y en que nuestro objetivo es clasificar y, que además no se requiere un learning rate alto ni complejizar el modelo. De hecho hacer ello hace que funcione de forma deficiente si lo medimos en base al accuracy que nos entrega. El learning rate ideal fue 0.01, como un punto medio entre un valor muy bajo 0.001 y muy alto 0.1. En este caso, se logró buena precisión del modelo para la clasificación y éste realizó los pasos adecuados y no saltos, como en el caso de 0.1 en que es muy rápido para capturar los detalles y por tanto, se los salta. Es así que un learning rate de 0.01 le otorga estabilidad al proceso y mejor clasificación. 

En el caso de probar la función de activación sigmoid con SGD, se puede observar que al juntar ambos el learning rate de mayor valor funciona mejor y, en este caso, el accuracy sería mayor. Por el contrario, al usar valores bajos como 0.001 se generan mayores pérdidas y baja accuracy. En los experimentos realizados con la función de activación de Tahn con SGD, pasa lo mismo que en el caso anterior en que al aumentar el learning rate el accuracy aumenta, pero el que cae mejor es el término medio de 0.01. 

Los mejores resultados se generaron con función de activación de Relu, ya que es más estable para lo que requiere este experimento de clasificar desde los datos MNIST. Se puede ver ello porque Relu funciona de buena forma tanto con SGD como con Adam

### 2.2 Redes convolucionales ###

In [86]:
# Verificar si hay una GPU disponible, de lo contrario usar la CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocesamiento: Definir transformaciones para los datos
transform = transforms.Compose([
    transforms.ToTensor(),                # Convertir imágenes a tensores
    transforms.Normalize((0.5,), (0.5,))  # Normalizar los valores a un rango de [-1, 1]
])

# Cargar el conjunto de datos MNIST
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)  # Datos de entrenamiento
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)  # Datos de prueba

# Crear DataLoaders para manejar los datos de forma eficiente
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)   # Loader para entrenamiento (batch de 128, mezclado)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)    # Loader para prueba (batch de 128, sin mezclar)

class CNN(nn.Module):
    def __init__(self, verbose=False, filters_l1=32, filters_l2=64, dropout=0.2, final_layer_size=128):
        super(CNN, self).__init__()
        self.verbose = verbose
        self.filters_l1 = filters_l1
        self.filters_l2 = filters_l2
        self.dropout_rate = dropout
        self.final_layer_size = final_layer_size

        # Primera capa convolucional
        self.conv1 = nn.Conv2d(1, self.filters_l1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Segunda capa convolucional
        self.conv2 = nn.Conv2d(self.filters_l1, self.filters_l2, kernel_size=3, stride=1, padding=1)

        # Calcular automáticamente las dimensiones de la capa lineal (fc1)
        self.fc1_input_size = self._calculate_fc1_input_size()
        
        # Primera capa completamente conectada
        self.fc1 = nn.Linear(self.fc1_input_size, self.final_layer_size)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc2 = nn.Linear(self.final_layer_size, 10)  # Capa de salida para 10 clases (MNIST)

    def _calculate_fc1_input_size(self):
        """
        Calcula automáticamente el tamaño de la entrada para la primera capa completamente conectada (fc1).
        Simula una pasada con una imagen de prueba de tamaño (1, 28, 28).
        """
        with torch.no_grad():  # Desactiva gradientes
            x = torch.randn(1, 1, 28, 28)  # Tensor ficticio de entrada con tamaño MNIST (batch_size=1)
            x = self.pool(torch.relu(self.conv1(x)))  # Aplicar Conv1 -> Pool
            x = self.pool(torch.relu(self.conv2(x)))  # Aplicar Conv2 -> Pool
            fc1_input_size = x.numel()  # Calcular número total de elementos
        return fc1_input_size

    def forward(self, x):
        if self.verbose: 
            print(f"Entrada: {x.shape}")  # Imprime la dimensión de la entrada

        # Primera capa convolucional, ReLU y MaxPooling
        x = self.pool(torch.relu(self.conv1(x)))
        if self.verbose:
            print(f"Después de Conv1 y MaxPooling: {x.shape}")  # Dimensión después de Conv1 y Pool

        # Segunda capa convolucional, ReLU y MaxPooling
        x = self.pool(torch.relu(self.conv2(x)))
        if self.verbose:
            print(f"Después de Conv2 y MaxPooling: {x.shape}")  # Dimensión después de Conv2 y Pool

        # Aplanar las características 2D a 1D
        x = x.view(-1, self.fc1_input_size)
        if self.verbose:
            print(f"Después de Aplanamiento: {x.shape}")  # Dimensión después de Flatten

        # Primera capa completamente conectada
        x = torch.relu(self.fc1(x))
        if self.verbose:
            print(f"Después de Fully Connected (fc1): {x.shape}")  # Dimensión después de fc1

        # Aplicar Dropout
        x = self.dropout(x)
        if self.verbose:
            print(f"Después de Dropout: {x.shape}")  # Dimensión después de Dropout

        # Capa de salida
        x = self.fc2(x)
        if self.verbose:
            print(f"Después de Fully Connected (fc2): {x.shape}")  # Dimensión después de fc2 (salida final)

        return x



In [ ]:
# Inicializar el modelo, la función de pérdida y el optimizador
model = CNN(verbose=False, filters_l1=8, filters_l2=32, dropout=0.2, final_layer_size=128).to(device)       # Mover el modelo a la GPU/CPU
criterion = nn.CrossEntropyLoss()                    # Función de pérdida para clasificación multiclase
optimizer = optim.Adam(model.parameters(), lr=0.001) # Optimizador Adam con tasa de aprendizaje 0.001

# Definir la función de entrenamiento
def train(model, loader, criterion, optimizer, device):
    model.train()  # Establecer el modelo en modo de entrenamiento
    running_loss = 0.0
    for images, labels in loader:  # Iterar sobre los lotes de datos
        images, labels = images.to(device), labels.to(device)  # Mover los datos a la GPU/CPU

        optimizer.zero_grad()       # Reiniciar los gradientes
        outputs = model(images)     # Paso hacia adelante
        loss = criterion(outputs, labels)  # Calcular la pérdida
        loss.backward()             # Paso hacia atrás (cálculo de gradientes)
        optimizer.step()            # Actualizar los pesos

        running_loss += loss.item()  # Acumular la pérdida
    return running_loss / len(loader)  # Devolver la pérdida promedio

# Definir la función de evaluación
def evaluate(model, loader, device):
    model.eval()  # Establecer el modelo en modo de evaluación
    correct = 0
    total = 0
    with torch.no_grad():  # Deshabilitar el cálculo de gradientes para ahorrar memoria
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)  # Mover datos a la GPU/CPU
            outputs = model(images)  # Paso hacia adelante
            _, predicted = torch.max(outputs, 1)  # Obtener las predicciones (clase con mayor probabilidad)
            total += labels.size(0)  # Contar el número total de ejemplos
            correct += (predicted == labels).sum().item()  # Contar las predicciones correctas
    return correct / total  # Calcular la precisión

# Bucle principal de entrenamiento
num_epochs = 10  # Número de épocas
for epoch in range(num_epochs):
    # Entrenar el modelo y calcular la pérdida
    train_loss = train(model, train_loader, criterion, optimizer, device)
    # Evaluar el modelo en el conjunto de prueba
    test_accuracy = evaluate(model, test_loader, device)
    # Imprimir los resultados de la época actual
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Calcular la precisión final en el conjunto de prueba
final_accuracy = evaluate(model, test_loader, device)
print(f"Final Test Accuracy: {final_accuracy:.4f}")

### Datos pruebas ###

RELU Y ADAM
En la arquitectura original  Accuracy: 0.9910. Filtros: filters_l1=8, filters_l2=32. Dropout 0.2, layer size 128. Learning rate 0.001
pruebas: 
RELU Y ADAM. l1 = 32, l2=64, l3=128, l4= 256, Dropout 0.4, layer size 256. LR 0.01. -> Accuracy 0.1135. Ahora son 4 capas convolucionales.
RELU Y ADAM. l1 = 32, l2=64, l3=128, l4= 256, Dropout 0.4, layer size 256. LR 0.1. -> Accuracy 0.009
l1 = 32, l2=64, l3=128, l4= 256, Dropout 0.3, layer size 256. LR 0.005. -> Accuracy 0.9862

Sigmoid y SGD -> l1 = 32, l2=64, l3=128, l4= 256, Dropout 0.4, layer size 256. LR 0.001.-> Accuracy 0.10
l1 = 64, l2=128, l3=256, l4= 512, Dropout 0.2, layer size 512. LR 0.001. -> Accuracy 0.1135
l1 = 64, l2=128, l3=256, l4= 512, Dropout 0.5, layer size 512. LR 0.1. -> Accuracy 0.0958

softplus y sgd -> l1 = 64, l2=128, l3=256, l4= 512, Dropout 0.2, layer size 512. LR 0.1. -> Accuracy 0.1135
l1 = 64, l2=128, l3=256, l4= 512, Dropout 0.2, layer size 512. LR 0.001. -> Accuracy 0.0980
l1=32, l2=64, l3=128, l4= 256, dropout=0.2, final_layer_size=256, LR 0.001. -> Accuracy 0.1135

Disminuir numeros capas. 
softplus y Adam -> l1=32, l2=64, l3=128, l4= 256, dropout=0.2, final_layer_size=256, LR 0.001. -> Accuracy 0.9875
l1=32, l2=64, l3=128, l4= 256, dropout=0.2, final_layer_size=256, LR 0.01.-> Accuracy 0.1135

softplus y RMSprOP -> l1=32, l2=64, l3=128, l4= 256, dropout=0.2, final_layer_size=256, LR 0.001.-> Accuracy 0.9824
l1 = 64, l2=128, l3=256, l4= 512, Dropout 0.2, layer size 512, LR 0.001.-> Accuracy 0.9743

Tahn y adam -> l1=32, l2=64, l3=128, l4= 256, dropout=0.2, final_layer_size=256, LR 0.001.-> Accuracy 0.9903
l1=32, l2=64, l3=128, l4= 256, dropout=0.1, final_layer_size=256, LR 0.01.-> Accuracy 0.1032
l1=8, l2=32, l3=64, l4= 128, dropout=0.2, final_layer_size=128, LR 0.005.-> Accuracy 0.9742
l1=8, l2=32, l3=64, l4= 128, dropout=0.2, final_layer_size=128, LR 0.001.-> Accuracy 0.9868

Tahn y SDG -> l1=32, l2=64, l3=128, l4= 256, dropout=0.2, final_layer_size=256, LR 0.001.-> Accuracy 0.9527
l1=32, l2=64, l3=128, l4= 256, dropout=0.3, final_layer_size=256, LR 0.005.-> Accuracy 0.9866


### Informe ###

La arquitectura original tenía 2 capas y además sus valores eran bajos, éstos fueron aumentado a término medio partiendo desde 32 y el rendimiento mejoró. Aún así con todos los experimentos realizados no se logró encontrar una arquitectura mejor en que el accuracy aumentara. Lo que sí se pudo observar es que al aumentar el valor de las capas y partir de valores altos como 64, esto generó que en la mayoria de los casos el accuracy fue muy bajo y por tanto, de rendimiento bajo. Por lo mismo, al principio no resultó mezclar las distintas funciones de activación con las funciones de optimización, pero el error fue  que los valores de las capas eran muy grandes y no compatibilizó con las características que tiene el modelo. En que no requiere de redes complejas ni varias capas, de hecho funciona de forma óptima con parámetros simples. Asimismo, luego de bajar los valores, todos los experimentos dieron un buen porcentaje de accuracy pero en el caso de aumentar el learning rate y que el optimizador fuera Adam, los valores de accuracy bajaron mucho. 

### Comparación ###

Tal como se mencionó anteriormente, el objetivo que quiere alcanzar este modelo con datos de tipo MNIST se logra no con un modelo que tenga mucha complejidad, de hecho al agregarle complejidad, aumentar el número de capas y el valor de los filtros el modelo no entregó resultados óptimos y por tanto, no rindió como queríamos. Respecto a los tiempos, pude notar que el tiempo aumentó si los filtros de las capas tenían un valor grande. Normalmente, tardó en 5 epochs 6 minutos y medio. En el caso de los experimentos con learning rate de 0.01 disminuyó a 4 minutos y medio. Con mayores capas el tiempo fue de 13 minutos y medio. Y, con la menor cantidad de capas y valor de los filtros el tiempo fue menor de 2 minutos y medio. En su mayoría se experimentó con 5 epochs para así lograr evaluar diferentes arquitecturas y parámetros de forma más rápida. 
